## The Yelp Dataset
[**The Yelp Dataset**](https://www.yelp.com/dataset_challenge/) is a dataset published by the business review service [Yelp](http://yelp.com) for academic research and educational purposes. I really like the Yelp dataset as a subject for machine learning and natural language processing demos, because it's big (but not so big that you need your own data center to process it), well-connected, and anyone can relate to it &mdash; it's largely about food, after all!

**Note:** If you'd like to execute this notebook interactively on your local machine, you'll need to download your own copy of the Yelp dataset. If you're reviewing a static copy of the notebook online, you can skip this step. Here's how to get the dataset:
1. Please visit the Yelp dataset webpage [here](https://www.yelp.com/dataset_challenge/)
1. Click "Get the Data"
1. Please review, agree to, and respect Yelp's terms of use!
1. The dataset downloads as a compressed .tgz file; uncompress it
1. Place the uncompressed dataset files (*yelp_academic_dataset_business.json*, etc.) in a directory named *yelp_dataset_challenge_academic_dataset*
1. Place the *yelp_dataset_challenge_academic_dataset* within the *data* directory in the *Modern NLP in Python* project folder

That's it! You're ready to go.

The files are text files (UTF-8) with one _json object_ per line, each one corresponding to an individual data record. 

In [ ]:
# !conda install -y tensorflow
# !conda install -y keras

In [1]:
import numpy as np
import os
import re
import json
from sklearn.model_selection import train_test_split
import codecs
import tensorflow as tf

SEED = 42
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()

The review records are stored in a similar manner &mdash; _key, value_ pairs containing information about the reviews.

In [2]:
json_dir = os.path.join('..', 'data',
                              'yelp_dataset_challenge_academic_dataset', 'dataset')

json_review_filepath = os.path.join(json_dir,
                                    'review.json')

with open(json_review_filepath, encoding='utf_8') as f:
    first_review_record = f.readline()
    
print(first_review_record)

{"review_id":"VfBHSwC5Vz_pbFluy07i9Q","user_id":"cjpdDjZyprfyDG3RlkVG3w","business_id":"uYHaNptLzDLoV_JZ_MuzUA","stars":5,"date":"2016-07-12","text":"My girlfriend and I stayed here for 3 nights and loved it. The location of this hotel and very decent price makes this an amazing deal. When you walk out the front door Scott Monument and Princes street are right in front of you, Edinburgh Castle and the Royal Mile is a 2 minute walk via a close right around the corner, and there are so many hidden gems nearby including Calton Hill and the newly opened Arches that made this location incredible.\n\nThe hotel itself was also very nice with a reasonably priced bar, very considerate staff, and small but comfortable rooms with excellent bathrooms and showers. Only two minor complaints are no telephones in room for room service (not a huge deal for us) and no AC in the room, but they have huge windows which can be fully opened. The staff were incredible though, letting us borrow umbrellas for t

A few attributes of note on the review records:
- __text__ &mdash; _the natural language text the user wrote_
- __stars__ &mdash; _the number of stars the reviewer left_

The _text_ and the _stars_ attribute will be our focus today!

In [3]:
sentiment_data_dir = os.path.join('..','data','sentiment_data')

text_filepath = os.path.join(sentiment_data_dir,'sentiment.txt')
sentiment_filepath = os.path.join(sentiment_data_dir,'number_of_stars.txt')



In [4]:
%%time
# Make the if statement True
# if you want to execute data prep yourself once you've got the yelp dataset saved.

if False:
    
    review_count = 0

    # create & open a new files in write mode
    with open(text_filepath, 'w', encoding='utf_8') as review_txt_file:
        with open(sentiment_filepath, 'w', encoding='utf_8') as review_sentiment_file:

            # open the existing review json file
            with open(json_review_filepath, encoding='utf_8') as review_json_file:
                # loop through all reviews in the existing file and convert to dict
                for review_json in review_json_file:
                    review = json.loads(review_json)
                    # write the review as a line in the new file
                    # escape newline characters in the original review text
                    review_txt_file.write(review.get('text','NA').replace('\n', r'\n') + '\n')
                    review_sentiment_file.write(str(review.get('stars','NA')) +'\n')
                    review_count =  review_count + 1

    print ('Text from {} reviews written to the new txt file.'.format(review_count))
    
else:
    
    with open(text_filepath, encoding='utf_8') as review_txt_file:
        for review_count, line in enumerate(review_txt_file):
            pass
        
    print('Text from {} reviews in the txt file.'.format(review_count + 1))



Text from 4739863 reviews in the txt file.
CPU times: user 9.08 s, sys: 1.63 s, total: 10.7 s
Wall time: 11.6 s


In [ ]:
# #count the lines in the above files

# from itertools import (takewhile,repeat)

# def rawincount(filename):
#     with open(filename, 'rb') as f:
#         bufgen = takewhile(lambda x: x, (f.raw.read(1024*1024) for _ in repeat(None)))
#         return sum( buf.count(b'\n') for buf in bufgen )

# print('Len of review text file:{}\nLen of review sentiment file:{}'.format(rawincount(review_txt_filepath), rawincount(review_sentiment_filepath)))

In [ ]:
# Good!  The lengths of the files match!

In [ ]:
#lets do train-test split


In [5]:
sentiment_train_dir = os.path.join(sentiment_data_dir, 'train')
sentiment_test_dir = os.path.join(sentiment_data_dir, 'test')

X_train_file_path = os.path.join(sentiment_train_dir, 'X_train.txt')
y_train_file_path = os.path.join(sentiment_train_dir, 'y_train.txt')
X_test_file_path = os.path.join(sentiment_test_dir, 'X_test.txt')
y_test_file_path = os.path.join(sentiment_test_dir, 'y_test.txt')

In [7]:
def find_sentiment(line): 
    ''' convert sentiment text (1-5 star rating) into positive or negative review'''
    try:
        if int(line.rstrip()) >= 3: #three stars or higher is positive review
            return 1
        else:
            return 0
    except:
        pass
    return "NA"

#Build subset of X and y in memory
num_rows = 30000
with open(sentiment_filepath, 'r') as f:
    y = [find_sentiment(line) for num,line in enumerate(f) if num < num_rows ]

with open(text_filepath, 'r') as f:
    X = [line.rstrip() for num,line in enumerate(f) if num < num_rows]

In [8]:
from collections import Counter

In [9]:
#check the sentiment ratings
count = Counter(y)
count

Counter({0: 6104, 1: 23896})

In [10]:
if True:
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=SEED)
else:
    pass


In [11]:
y_train_counter = Counter(y_train)
y_test_counter = Counter(y_test)

print(y_train_counter, y_train_counter)

Counter({1: 19136, 0: 4864}) Counter({1: 19136, 0: 4864})


In [13]:
!conda install -y pandas

Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /anaconda:

The following packages will be UPDATED:

    pandas: 0.20.3-py36_0 --> 0.21.0-py36_0 conda-forge

pandas-0.21.0- 100% |################################| Time: 0:00:03   3.08 MB/s


In [18]:
!conda update -y dask

Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /anaconda:

The following NEW packages will be INSTALLED:

    dask-core:        0.15.4-py_0   conda-forge
    distributed:      1.19.3-py36_0 conda-forge
    msgpack-python:   0.4.8-py36_0  conda-forge
    sortedcontainers: 1.5.7-py36_0  conda-forge
    tblib:            1.3.2-py36_0  conda-forge
    zict:             0.1.3-py_0    conda-forge

The following packages will be UPDATED:

    bokeh:            0.12.5-py36_0             --> 0.12.10-py36_0 conda-forge
    dask:             0.13.0-py36_0             --> 0.15.4-py_0    conda-forge
    partd:            0.3.7-py36_0              --> 0.3.8-py36_0   conda-forge
    tornado:          4.4.2-py36_0              --> 4.5.2-py36_0   conda-forge

dask-core-0.15 100% |################################| Time: 0:00:00   1.92 MB/s
msgpack-python 100% |################################| Time: 0:00:00   1.91 MB/s
sortedcont

In [19]:
import keras
keras.__version__

'2.0.6'

In [20]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM


In [ ]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #print string
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", "", string)
    string = re.sub(r"\)", "", string)
    string = re.sub(r"\?", "", string)
    string = re.sub(r"/", "", string)
    string = re.sub(r"\s{2,}", " ", string)

    return string.strip().lower()